### Classificador de pagamentos em boleto
#### Versão 1: KNN

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

#### Parte 1: Preparando o Dataset

In [3]:
billet_dataset = pd.read_csv('billet_dataset.csv',  
                         engine='python', 
                         sep=r'\s*,\s*')

In [4]:
billet_dataset.head()

,payment_creation_date_id,payment_creation_date,payment_creation_part_of_day,payment_creation_year,payment_creation_month,payment_creation_day,payment_creation_week_day,payment_creation_year_week,payment_creation_year_quarter,payment_creation_year_half,...,brand,store,sales_channel,device,utm_source,utm_medium,utm_campaign,utm_term,utm_content,is_paid
0,1704,2018-08-31,morning,2018,8,31,6,35,3,2,...,BeYoung,"""BeYoung - Booster""",televendas,televendas,200.223.176.42,televendas,"""Paloma Oliveira Cunha da Silva""",televendas,televendas,0
1,1704,2018-08-31,dawn,2018,8,31,6,35,3,2,...,BeYoung,"""BeYoung - Booster""",website,Desktop,EmailMarketing_News,CRM,BYM_20180830_009_perf10sf_ativos_com_boleto,BB00,NaN,1
2,1704,2018-08-31,night,2018,8,31,6,35,3,2,...,BeYoung,"""BeYoung - Booster""",website,Desktop,EmailMarketing_News,CRM,Recuperacao_Cart_com_boleto,NaN,NaN,0
3,1704,2018-08-31,morning,2018,8,31,6,35,3,2,...,Slimcaps,Slimcaps,website,Desktop,ALLINMAIL,email,SC_20180831_0178_perf20sf_ativos_com_boleto,__.qbv.nm.lsl.w.ytd.g.q2.u.t.yu.f.waj.v.hbz.v....,235244994,0
4,1704,2018-08-31,morning,2018,8,31,6,35,3,2,...,BeYoung,"""BeYoung - Booster""",website,Mobile,EmailMarketing_News,CRM,BYB_20180831_0103_perf25sf_ativos_com_boleto,BB06,NaN,0


In [5]:
billet_dataset.describe()

,payment_creation_date_id,payment_creation_year,payment_creation_month,payment_creation_day,payment_creation_week_day,payment_creation_year_week,payment_creation_year_quarter,payment_creation_year_half,payment_creation_is_workday,is_paid
count,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000,374539.000000
mean,1402.431493,2017.346290,6.441748,15.463853,3.953441,26.185759,2.486339,1.489370,0.768088,0.324527
std,159.059045,0.475788,2.959034,8.995487,1.861660,12.934716,1.002646,0.499888,0.422054,0.468198
min,1128.000000,2017.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1270.000000,2017.000000,4.000000,7.000000,2.000000,16.000000,2.000000,1.000000,1.000000,0.000000
50%,1383.000000,2017.000000,6.000000,15.000000,4.000000,26.000000,2.000000,1.000000,1.000000,0.000000
75%,1534.000000,2018.000000,9.000000,23.000000,5.000000,36.000000,3.000000,2.000000,1.000000,1.000000
max,1704.000000,2018.000000,12.000000,31.000000,7.000000,52.000000,4.000000,2.000000,1.000000,1.000000


In [6]:
billet_dataset.dtypes

payment_creation_date_id          int64
payment_creation_date            object
payment_creation_part_of_day     object
payment_creation_year             int64
payment_creation_month            int64
payment_creation_day              int64
payment_creation_week_day         int64
payment_creation_year_week        int64
payment_creation_year_quarter     int64
payment_creation_year_half        int64
payment_creation_is_workday       int64
brand                            object
store                            object
sales_channel                    object
device                           object
utm_source                       object
utm_medium                       object
utm_campaign                     object
utm_term                         object
utm_content                      object
is_paid                           int64
dtype: object

In [7]:
billet_dataset.isnull().any()

payment_creation_date_id         False
payment_creation_date            False
payment_creation_part_of_day     False
payment_creation_year            False
payment_creation_month           False
payment_creation_day             False
payment_creation_week_day        False
payment_creation_year_week       False
payment_creation_year_quarter    False
payment_creation_year_half       False
payment_creation_is_workday      False
brand                            False
store                            False
sales_channel                    False
device                           False
utm_source                       False
utm_medium                        True
utm_campaign                      True
utm_term                          True
utm_content                       True
is_paid                          False
dtype: bool

In [8]:
billet_dataset = billet_dataset.fillna('')

In [9]:
billet_dataset.shape

(374539, 21)

In [10]:
billet_dataset['payment_creation_date'] = pd.to_datetime(billet_dataset['payment_creation_date'])

In [11]:
Xfields = ['payment_creation_part_of_day', 'payment_creation_year',
           'payment_creation_month', 'payment_creation_day',
           'payment_creation_week_day', 'payment_creation_year_week',
           'payment_creation_year_quarter', 'payment_creation_year_half',
           'payment_creation_is_workday', 'brand', 'store', 'sales_channel',
           'device', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_term',
           'utm_content']

train_data = billet_dataset.loc[billet_dataset['payment_creation_date']<'2018-08-01']
test_data = billet_dataset.loc[billet_dataset['payment_creation_date']>='2018-08-01']
Xtrain_data = train_data[Xfields]
Ytrain_data = train_data.is_paid
Xtest_data = test_data[Xfields]
Ytest_data = test_data.is_paid

#### Parte 2: Label Encoding

In [12]:
from sklearn import preprocessing

In [13]:
Xtrain_data = Xtrain_data.apply(preprocessing.LabelEncoder().fit_transform)
Xtest_data = Xtest_data.apply(preprocessing.LabelEncoder().fit_transform)

In [14]:
Xtrain_data.shape

(356409, 18)

In [15]:
Xtrain_data.head()

,payment_creation_part_of_day,payment_creation_year,payment_creation_month,payment_creation_day,payment_creation_week_day,payment_creation_year_week,payment_creation_year_quarter,payment_creation_year_half,payment_creation_is_workday,brand,store,sales_channel,device,utm_source,utm_medium,utm_campaign,utm_term,utm_content
18130,2,1,6,30,2,30,2,1,1,4,7,2,2,29,8,449,511,0
18131,3,1,6,30,2,30,2,1,1,1,2,1,4,18,32,4,3828,3940
18132,0,1,6,30,2,30,2,1,1,1,2,1,4,18,32,76,3828,3940
18133,0,1,6,30,2,30,2,1,1,4,7,2,2,41,21,2055,831,0
18134,2,1,6,30,2,30,2,1,1,1,2,2,2,33,21,1418,1672,0


#### Parte 3: Análise de Correlação

In [17]:
analise_dataset = Xtrain_data
analise_dataset['is_paid'] = Ytrain_data
analise = analise_dataset.corr().loc[:,'is_paid'].sort_values(ascending=True).dropna()
analise

sales_channel                   -0.189924
store                           -0.097222
brand                           -0.084295
utm_campaign                    -0.081457
payment_creation_year_quarter   -0.049099
payment_creation_year_week      -0.044343
payment_creation_month          -0.044304
payment_creation_year_half      -0.043531
utm_medium                      -0.025632
payment_creation_part_of_day    -0.021691
payment_creation_day            -0.009618
payment_creation_week_day       -0.008862
utm_term                         0.000405
utm_source                       0.039748
utm_content                      0.052672
payment_creation_year            0.055482
payment_creation_is_workday      0.090714
device                           0.116021
is_paid                          1.000000
Name: is_paid, dtype: float64

A correlação entre os campos e a váriável de classe é pequena, como é possível notar no gráfico acima.

#### Parte 4: Model Fitting

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [19]:
knn = KNeighborsClassifier(n_neighbors=3)

In [20]:
scores = cross_val_score(knn, Xtrain_data, Ytrain_data, cv=10)

In [21]:
scores.mean()

0.5292608395239552